![page1](./image/page1.png)

# 前言 - 学习本项目你可以获得什么
- 理论学习：了解AIAgent的基础知识
- 上手实操：深入了解Agent中的FunctionCall运行流程
- 上手实操：入门百度智能云千帆AppBuilder，在十分钟内打造一个个性化AIAgent
- 上手实操：使用AppBuilder-SDK打造一个端云组件联动的进阶Agent

## 1. 项目背景

### 1.1、 什么是AppBuilder
[百度智能云千帆AppBuilder](https://appbuilder.cloud.baidu.com/)（以下简称AppBuilder）是基于大模型搭建AI原生应用的工作台，旨在降低AI原生应用的开发门槛，赋能开发者和企业快速实现应用搭建。

平台提供了RAG（检索增强生成）、Agent（智能体）等应用框架，内置了文档问答、表格问答、多轮对话、生成创作等多种应用组件，还包括百度搜索和百度地图等特色组件，以及文本处理、图像处理和语音处理等传统AI组件，支持零代码、低代码、全代码三种开发方式，满足不同开发能力的开发者和企业的场景需求。

### 1.2、 什么是AppBuilder-SDK

[百度智能云千帆AppBuilder-SDK](https://github.com/baidubce/app-builder)(以下简称AB-SDK)，百度智能云千帆AppBuilder-SDK是百度智能云千帆AppBuilder面向AI原生应用开发者提供的一站式开发平台的客户端SDK。

![page2](./image/page2.png)

# 2. 项目介绍 - 通过ToolCall实现端云组件联动的Agent
### 2.1、 什么是Agent

AIAgent是能够感知环境，基于目标进行决策并执行动作的智能化应用。不同于传统人工智能应用（主要指以规则引擎、机器学习、深度学习等技术为核心）和RPA机器人，AIAgent能够基于目标和对现状能力的认知，在环境约束中，依赖特定资源和现有工具，找到行动规则并将行动拆解为必要的步骤，自主执行步骤，达成目标。

AIAgent具备三个核心能力：独立思考、自主执行、持续迭代。
- 独立思考是指AlAgent能够根据给定任务目标和约束条件，进行任务规划和问题拆解，形成执行步骤（即工作流）；
- 自主执行是指AlAgent能够调取各类组件和工具，按照执行步骤依次执行，实现任务目标；
- 持续选代是指AlAgent能够自动记录任务目标、工作流和执行结果，基于结果反馈，沉淀专家知识和案例。

AICopilot、AIAgent、大模型等名词在各类文章上经常混淆，此处简要说明下三者的区别。大模型一般是指大模型技术，AlAgent和Al Copilot是基于大模型技术的智能化应用，AlAgent和AlCopilot在功能和场景上存在差别。

自主性是AIAgent和AI Copilot之间最大的区别。AI Copilot是“副驾驶”，只是提供建议而非决策，AIAgent是“主驾驶”，需要真正做出决策并开展行动。

![page3](./image/page3.png)

### 2.2、 什么是ToolCall

AIAgent 有四大核心组件：记忆、规划、工具和执行。其中工具部分，与我们的开发关系最密切，在各类Agent开发平台/工具中，常被称为“组件”、"插件"、"能力"等.

我们可以学习关于Agent工具的定义与分类，如下图~

![page4](./image/page4.png)

### 2.3、 什么是端云组件联动

这个话题，是从工具的执行位置出发展开的~ 使用如AppBuilder / Coze 等平台开发Agent时，我们可以使用很多